In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
import seaborn as sns
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
!python -m spacy download el_core_news_sm

2023-07-16 15:51:30.837937: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 33.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')


In [3]:
nlp = spacy.load('el_core_news_sm')

In [4]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [5]:
start_url = 'https://www.pronews.gr/search/%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82'

In [6]:
response = requests.get(start_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [7]:
teaser_articles_list = doc.find('div', {'class': 'columns is-variable is-3-desktop is-4-widescreen is-6-fullhd is-multiline is-mobile'}).find_all('div', {'class': 'column is-half-desktop is-half-tablet is-full-mobile'})

In [8]:
teaser_articles_list

[<div class="column is-half-desktop is-half-tablet is-full-mobile"><article class="pt-post post-article prel pinpost" id="article-2826525"><figure class="post-figure prel"> <a class="hrefattribute thumb-holder ratio-16-9" href="https://www.pronews.gr/elliniki-politiki/gia-na-min-ksexniomaste-antiemvoliastes-rosofiloi-kai-psekasmenoi-den-symfonoun-me-tin-politiki-mas-elege-i-n-mpakogianni/"> <noscript><img alt="Για να μην ξεχνιόμαστε: «Αντιεμβολιαστές, ρωσόφιλοι και ψεκασμένοι δεν συμφωνούν με την πολιτική μας» έλεγε η Ν.Μπακογιάννη" class="imagefit" height="438" loading="lazy" src="https://www.pronews.gr/wp-content/uploads/2023/01/24/ntora_bakogianni1.jpg" width="780"/></noscript><img alt="Για να μην ξεχνιόμαστε: «Αντιεμβολιαστές, ρωσόφιλοι και ψεκασμένοι δεν συμφωνούν με την πολιτική μας» έλεγε η Ν.Μπακογιάννη" class="lazyload imagefit" data-src="https://www.pronews.gr/wp-content/uploads/2023/01/24/ntora_bakogianni1.jpg" height="438" loading="lazy" src="data:image/svg+xml,%3Csvg%20xml

In [9]:
link = teaser_articles_list[0].find('figure', {'class': 'post-figure prel'}).find('a')['href']

link

'https://www.pronews.gr/elliniki-politiki/gia-na-min-ksexniomaste-antiemvoliastes-rosofiloi-kai-psekasmenoi-den-symfonoun-me-tin-politiki-mas-elege-i-n-mpakogianni/'

In [10]:
teasers_list = []

response = requests.get(start_url)
doc = BeautifulSoup(response.text, 'html.parser')

#*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo teaser_articles_list***
teaser_articles_list = doc.find('div', {'class': 'columns is-variable is-3-desktop is-4-widescreen is-6-fullhd is-multiline is-mobile'}).find_all('div', {'class': 'column is-half-desktop is-half-tablet is-full-mobile'})


for teaser in teaser_articles_list:
    story_dict = {}
    try:

      #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στο link, ΜΕΤΑ το teaser
      link = teaser.find('figure', {'class': 'post-figure prel'}).find('a')['href']


      if link.startswith("http"):
        story_dict['url'] = link
      else:
        story_dict['url'] = main_url + link
    except:
      story_dict['url'] = ''
      pass
    teasers_list.append(story_dict)
time.sleep(1)

In [11]:
pronews_teasers_df = pd.DataFrame(teasers_list)
pronews_teasers_df

,url
0,https://www.pronews.gr/elliniki-politiki/gia-n...
1,https://www.pronews.gr/koinonia/thanos-pleyris...
2,https://www.pronews.gr/elliniki-politiki/nd/n-...
3,https://www.pronews.gr/elliniki-politiki/10585...
4,https://www.pronews.gr/elliniki-politiki/10510...
5,https://www.pronews.gr/elliniki-politiki/10470...
6,https://www.pronews.gr/elliniki-politiki/10415...
7,https://www.pronews.gr/kosmos/1039414_rosia-st...
8,https://www.pronews.gr/elliniki-politiki/10381...
9,https://www.pronews.gr/thriskeia/1032730_eisag...


In [12]:
pronews_teasers_df.to_csv('/content/temp/pronews_teasers.csv', index=False)

In [13]:
article_url = 'https://www.pronews.gr/thriskeia/1016304_orthodoxa-hristianika-somateia-den-eimaste-antiemvoliastes-kathe/'

In [14]:
response = requests.get(article_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [15]:
article = doc.find('div', {'id': 'website'})

In [16]:
section = article.find('a', {'class': 'progray uppercase'}).text
section

'Θρησκεία '

In [17]:
title = article.find('h1', {'class': 'post-title zonabold article-hero'}).text
title

' Ορθόδοξα Χριστιανικά Σωματεία: «Δεν είμαστε αντιεμβολιαστές – Κάθε ιατρική πράξη πρέπει να αποτελεί ελεύθερη επιλογή»'

In [18]:
author = article.find('span', {'class': 'author-name'}).text
author

'Newsroom'

In [19]:
date = article.find('time')['datetime']
date

'2021.09.10 10:36'

In [20]:
paragraphs = article.find('div', {'class': 'csscontent wrap-content-body field-item even'}).find_all('p')
paragraphs

[<p><strong>Τα Ορθόδοξα Χριστιανικά Σωματεία των Αθηνών τόνισαν ότι δεν ανήκουν σε κάποιο «αντιεμβολιαστικό» κίνημα, ενώ στάθηκαν στο πλευρό των εργαζόμενων στους τομείς της Υγείας και Πρόνοιας που μπήκαν σε υποχρεωτικό καθεστώς αναστολής εργασίας άνευ αποδοχών.</strong></p>,
 <p>Δεν ανήκουμε σε κάποιο «αντιεμβολιαστικό» κίνημα, ούτε  είμαστε κατά των εμβολίων, γενικά.</p>,
 <p><em>Δηλώνουμε όμως ότι, ο εμβολιασμός, όπως και κάθε άλλη ιατρική πράξη, πρέπει να αποτελούν ελεύθερη επιλογή του κάθε ανθρώπου και οποιοσδήποτε εκβιασμός για την πραγματοποίησή τους, σε συνδυασμό με την μη ανάληψη ιατρικής και νομικής ευθύνης από την Πολιτεία αλλά και από τα αρμόδια ιατρικά όργανα, για τις παρενέργειες από τα εμβόλια αυτά, δεν συνάδουν ούτε με τις αρχές της ιατρικής επιστήμης, ούτε με το νομικό πολιτισμό μας, το Σύνταγμα, τις Διεθνείς Συμβάσεις Δικαιωμάτων του Ανθρώπου, ούτε με την χριστιανική πίστη που κατατάσσει την ελευθερία και το αυτεξούσιο ως σύμφυτες ιδιότητες της ουσίας του ανθρώπου. Εξ

In [21]:
p_texts_list = []                      # δημιουργία κενής λίστας για να αποθηκεύσουμε το text κάθε παραγράφου
paragraphs = article.find('div', {'class': 'csscontent wrap-content-body field-item even'}).find_all('p')     # λίστα με όλες τις παραγράφους
for p in paragraphs:                   # για κάθε μια από τις παραγράφους στη λίστα
  p_texts_list.append(p.text)          # πάρε το .text και πρόσθεσέ το στη λίστα με τα texts των παραγράφων
full_text = ' '.join(p_texts_list)     # συνένωση των texts των παραγράφων
full_text = "".join(full_text.splitlines())
full_text

'Τα Ορθόδοξα Χριστιανικά Σωματεία των Αθηνών τόνισαν ότι δεν ανήκουν σε κάποιο «αντιεμβολιαστικό» κίνημα, ενώ στάθηκαν στο πλευρό των εργαζόμενων στους τομείς της Υγείας και Πρόνοιας που μπήκαν σε υποχρεωτικό καθεστώς αναστολής εργασίας άνευ αποδοχών. Δεν ανήκουμε σε κάποιο «αντιεμβολιαστικό» κίνημα, ούτε  είμαστε κατά των εμβολίων, γενικά. Δηλώνουμε όμως ότι, ο εμβολιασμός, όπως και κάθε άλλη ιατρική πράξη, πρέπει να αποτελούν ελεύθερη επιλογή του κάθε ανθρώπου και οποιοσδήποτε εκβιασμός για την πραγματοποίησή τους, σε συνδυασμό με την μη ανάληψη ιατρικής και νομικής ευθύνης από την Πολιτεία αλλά και από τα αρμόδια ιατρικά όργανα, για τις παρενέργειες από τα εμβόλια αυτά, δεν συνάδουν ούτε με τις αρχές της ιατρικής επιστήμης, ούτε με το νομικό πολιτισμό μας, το Σύνταγμα, τις Διεθνείς Συμβάσεις Δικαιωμάτων του Ανθρώπου, ούτε με την χριστιανική πίστη που κατατάσσει την ελευθερία και το αυτεξούσιο ως σύμφυτες ιδιότητες της ουσίας του ανθρώπου. Εξ άλλου δεν υπάρχει επιστημονική τεκμηρίωση

In [22]:
full_articles_list = []

for article_url in pronews_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo article που βρήκατε παραπάνω***
  article = doc.find('div', {'id': 'website'})

  full_article_dict = {}

  # website
  full_article_dict['site'] = "pronews.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = article.find('a', {'class': 'progray uppercase'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = article.find('h1', {'class': 'post-title zonabold article-hero'}).text
    full_article_dict['title'] = title
  except:
    try:
      title = article.find('h1', {'class': 'post-title zonabold pure-black-c article-hero'}).text
      full_article_dict['title'] = title
    except:
      full_article_dict['title'] = ''
      pass

  # date
  try:
    date = article.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  # author
  try:
    author = article.find('span', {'class': 'author-name'}).text
    full_article_dict['author'] = author
  except:
    try:
      author = article.find('div', {'class': 'article__blogger'}).text
      full_article_dict['author'] = author
    except:
      full_article_dict['author'] = ''
      pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find('div', {'class': 'csscontent wrap-content-body field-item even'}).find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)

In [23]:
pronews_full_articles_df = pd.DataFrame(full_articles_list)
pronews_full_articles_df

,site,url,section,title,date,author,full_text
0,pronews.gr,https://www.pronews.gr/elliniki-politiki/gia-n...,Ελληνική Πολιτική,"Για να μην ξεχνιόμαστε: «Αντιεμβολιαστές, ρωσ...",2023.01.24 14:38,Newsroom,Δεν έχει περάσει πολύς καιρός από τότε που η π...
1,pronews.gr,https://www.pronews.gr/koinonia/thanos-pleyris...,Κοινωνία,Θάνος Πλεύρης: «Έξω από το σπίτι μου έχουν συ...,2022.06.27 20:19,Newsroom,Αντιεμβολιαστές διαδηλώνουν έξω από την κατοικ...
2,pronews.gr,https://www.pronews.gr/elliniki-politiki/nd/n-...,ΝΔ,Ν.Μπακογιάννη: «Η ΝΔ δεν θα χάσει ποσοστά για...,2022.05.04 11:08,Newsroom,"Η βουλευτής της Νέας Δημοκρατίας, Ντόρα Μπακογ..."
3,pronews.gr,https://www.pronews.gr/elliniki-politiki/10585...,Κυβέρνηση,Θ.Πλεύρης: «Ο ΣΥΡΙΖΑ υιοθετεί χωρίς προσχήμα...,2022.02.17 09:43,Newsroom,"Ο υπουργός Υγείας Θάνος Πλεύρης, απαντώντας στ..."
4,pronews.gr,https://www.pronews.gr/elliniki-politiki/10510...,Ελληνική Πολιτική,"Βουλευτής ΝΔ Σ.Κελέτσης: «Ακροδεξιοί, αντιεμβ...",2022.01.21 09:22,Newsroom,Στο ερώτημα των κατοίκων γιατί δεν πήγαν οι βο...
5,pronews.gr,https://www.pronews.gr/elliniki-politiki/10470...,Ελληνική Πολιτική,«Εμφύλιος» στην Εκκλησία – Μητροπολίτης Κορίν...,2022.01.05 15:43,Newsroom,"Ο Μητροπολίτης Κορίνθου Διονύσιος, μιλώντας σε..."
6,pronews.gr,https://www.pronews.gr/elliniki-politiki/10415...,Ελληνική Πολιτική,Γ.Πετράκος για Γ.Τράγκα: «Ακόμα βλέπουν φαντά...,2021.12.14 11:36,Newsroom,Ο αναπληρωτής Γραμματέας του Κινήματος Ελεύθερ...
7,pronews.gr,https://www.pronews.gr/kosmos/1039414_rosia-st...,Κόσμος,Ρωσία: Εξοντώνουν τον ρωσικό λαό λέει εκπρόσω...,2021.12.06 17:04,Newsroom,Τα όσα ισχυρίζονται εκείνοι που αρνούνται την ...
8,pronews.gr,https://www.pronews.gr/elliniki-politiki/10381...,Ελληνική Πολιτική,Πολάκης σε πρωθυπουργό: «Είσαι άσχετος και επ...,2021.12.01 19:51,Newsroom,«Άσχετο και επικίνδυνο» χαρακτήρισε τον Κυριάκ...
9,pronews.gr,https://www.pronews.gr/thriskeia/1032730_eisag...,Θρησκεία,Εισαγγελική παρέμβαση για αντιεμβολιαστές μον...,2021.11.10 14:31,Newsroom,Εισαγγελική παρέμβαση προκάλεσαν δηλώσεις του ...


In [24]:
pronews_full_articles_df.to_csv('/content/temp/pronews_full_articles.csv', index=False)